## Introduction

* pipeline: 데이터 전처리 코드와 모델링 코드를 정리하는 방법
* pipeline의 장점
    1. 깔끔한 코드: 전처리의 각 스텝에서 데이터를 설명하는 것은 지저분할 수 있음. pipeline을 사용하면, 각 스텝에서 training data와 validation data를 일일이 찾아볼 필요가 없음
    2. 적은 버그: 전처리 과정을 빠트리거나 잘못 적용할 위험이 적음
    3. 더 쉬운 일반화: 프로토타입을 대규모로 배포하기 위한 모델로 변경하는 것은 어려울 수 있음. but 변경 과정에서 생길 수 있는 문제에 pipeline이 도움이 될 수 있음
    4. 모델 검증을 위한 더 많은 옵션들: ex. cross-validation
    
<br>

## Example


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


* 전체 pipeline을 3단계로 구성

<br>

#### Step 1: Define Preprocessing Steps

* `ColumnTransformer`: 전처리 과정을 하나로 묶기 위해 사용하는 클래스

<br>

* 숫자 데이터 중 missing value에 imputation 처리
* categorical data의 missing value에 imputation, one-hot encoding 처리

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

## Preprocessing for numerical data ##
numerical_transformer = SimpleImputer(strategy='constant')

## Preprocessing for categorical data ##
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                         ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                         ])

## Bundle preprocessing for numerical and categorical data ##
preprocessor = ColumnTransformer(
transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

#### Step 2: Define the Model

* RandomForest 모델 정의

In [7]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

#### Step 3: Create and Evaluate the Pipeline

* 전처리와 모델링을 pipeline으로 정의하는 데 중요한 포인트
    * 학습 데이터의 전처리와 모델 fitting을 한 줄로 작성(pipeline을 사용하지 않으면, imputation, one-hot encoding, 모델 학습을 각각의 과정으로 해야함. 숫자 데이터와 categorical data를 모두 처리해야하는 경우엔 코드가 지저분해짐)
    * `predict()`를 사용할 때 **X_valid**에 전처리를 하지 않음. pipeline은 예측하기 전에 자동으로 X_valid에 전처리를 수행함(pipeline을 사용하지 않으면, validation data 전처리를 기억하고 있어야함)

In [8]:
from sklearn.metrics import mean_absolute_error

## Bundle preprocessing and modeling code in a pileline ##
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model', model)])

## Preprocessing of training data, fit model ##
my_pipeline.fit(X_train, y_train)

## Preprocessing of validation data, get predictions ##
preds = my_pipeline.predict(X_valid)

## Evaluate the model ##
score = mean_absolute_error(y_valid, preds)
print('MAE: ', score)

MAE:  160679.18917034855


<br>

## Conclusion

* pipeline은 머신러닝 코드를 깔끔하게 해주고, 오류를 피할 수 있게 해줌
* 아주 복잡한 데이터 전처리 과정을 정리하는데 유용함